#### Setup

In [5]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import keras
import tensorflow as tf
import numpy as np 
from keras import layers

#### Load the data: IMDB movie review sentiment classification
- データをダウンロードして構造を見る

In [6]:
# !curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
# !tar -xf aclImdb_v1.tar.gz

In [7]:
!ls aclImdb
!ls aclImdb/test
!ls aclImdb/train

README
imdb.vocab
imdbEr.txt
test
train
labeledBow.feat
neg
pos
urls_neg.txt
urls_pos.txt
labeledBow.feat
neg
pos
unsupBow.feat
urls_neg.txt
urls_pos.txt
urls_unsup.txt


In [8]:
# aclImdb/train/pos および aclImdb/train/negフォルダにはテキストファイルがあり、それぞれ1つのレビュー（肯定的または否定的）を表す：
!cat aclImdb/train/pos/6248_7.txt

Being an Austrian myself this has been a straight knock in my face. Fortunately I don't live nowhere near the place where this movie takes place but unfortunately it portrays everything that the rest of Austria hates about Viennese people (or people close to that region). And it is very easy to read that this is exactly the directors intention: to let your head sink into your hands and say "Oh my god, how can THAT be possible!". No, not with me, the (in my opinion) totally exaggerated uncensored swinger club scene is not necessary, I watch porn, sure, but in this context I was rather disgusted than put in the right context.<br /><br />This movie tells a story about how misled people who suffer from lack of education or bad company try to survive and live in a world of redundancy and boring horizons. A girl who is treated like a whore by her super-jealous boyfriend (and still keeps coming back), a female teacher who discovers her masochism by putting the life of her super-cruel "lover" 

In [9]:
# posとnegのサブフォルダーにしか興味がないので、テキストファイルが入っている他のサブフォルダーを削除しよう：
!rm -r aclImdb/train/unsup

rm: cannot remove 'aclImdb/train/unsup': No such file or directory


- <span style="color: red;">keras.utils.text_dataset_from_directory</span>ユーティリティを使用すると、クラス固有のフォルダにファイリングされたディスク上のテキストファイルのセットから、ラベル付きの<span style="color: red;">tf.data.Dataset</span>オブジェクトを生成できます。
- これを使って、トレーニング、検証、テストのデータセットを生成してみよう。検証用データセットとトレーニング用データセットは、<span style="color: red;">train</span>ディレクトリの2つのサブセットから生成され、サンプルの20%が検証用データセットに、80%がトレーニング用データセットになります。
  
- テストデータセットに加えて検証データセットを持つことは、テストデータセットを使うべきでないモデルアーキテクチャなどのハイパーパラメータをチューニングするのに便利である。
- しかし、モデルを実世界に出す前に、（検証データセットを作成せずに）利用可能なすべての訓練データを使って再学習させ、その性能を最大にする必要がある。
- <span style="color: red;">validation_split</span>と<span style="color: red;">subset</span>引数を使う場合は、ランダムなシードを指定するか、<span style="color: red;">shuffle=False</span>を渡すようにして、検証用とトレーニング用の分割が重ならないようにしてください。

In [10]:
batch_size = 32
raw_train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=1337,
)
raw_val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="validation",
    seed=1337,
)
raw_test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test",
    batch_size=batch_size,
)

print(f"Number of batches in raw_train_ds: {raw_train_ds.cardinality()}")
print(f"Number of batches in raw_val_ds: {raw_val_ds.cardinality()}")
print(f"Number of batches in raw_test_ds: {raw_test_ds.cardinality()}")

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.
Number of batches in raw_train_ds: 625
Number of batches in raw_val_ds: 157
Number of batches in raw_test_ds: 782


- いくつかサンプルを見てみる
    - 正規化とトークン化が期待通りに機能しているかどうか確認するために、生データをチェックすることは大切。

In [11]:
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(10):
        print(text_batch.numpy()[i])
        print(label_batch.numpy()[i])

b'I\'ve seen tons of science fiction from the 70s; some horrendously bad, and others thought provoking and truly frightening. Soylent Green fits into the latter category. Yes, at times it\'s a little campy, and yes, the furniture is good for a giggle or two, but some of the film seems awfully prescient. Here we have a film, 9 years before Blade Runner, that dares to imagine the future as somthing dark, scary, and nihilistic. Both Charlton Heston and Edward G. Robinson fare far better in this than The Ten Commandments, and Robinson\'s assisted-suicide scene is creepily prescient of Kevorkian and his ilk. Some of the attitudes are dated (can you imagine a filmmaker getting away with the "women as furniture" concept in our oh-so-politically-correct-90s?), but it\'s rare to find a film from the Me Decade that actually can make you think. This is one I\'d love to see on the big screen, because even in a widescreen presentation, I don\'t think the overall scope of this film would receive its

#### Prepare the data
- < /br>タグを削除する

In [12]:
import string
import re

# 上記のデータを見ると、生のテキストには'<br />'形式のHTMLタグが含まれている
# これらのタグはデフォルトの標準化機能では除去されない
# そのため、カスタム標準化関数を作成する
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', " ")
    return tf.strings.regex_replace(
        stripped_html, f"[{re.escape(string.punctuation)}]", ""
    )

# 定数
max_features = 20000
embedding_dim = 128
sequence_length = 500

# カスタム標準化ができたので、テキスト・ベクタ化レイヤーをインストールできる。
# このレイヤーを使って文字列の正規化、分割、整数へのマッピングを行うので、"output_mode"を"int"に設定する。
# デフォルトの分割関数と、上記で定義したカスタム標準化を使用していることに注意する
# また、配列の最大長を明示的に設定する。後述するCNNモデルでは、乱雑な配列はサポートされないから。
vectorize_layer = keras.layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)

# vectorize_layerが作成されたので、テキストだけのデータセットに対して、"adapt"を呼び出して、語彙を作成する
# バッチ処理をする必要はないが、非常に大きなデータセットの場合、これはデータセットの予備コピーをメモリに保存しないようにする。

# では、テキストだけのデータセット（ラベルなし）を作ってみる
text_ds = raw_train_ds.map(lambda x, y: x)
# "adapt"を呼び出す
vectorize_layer.adapt(text_ds)

#### Two options to vectorize the data
- text_vectorization_layerの使い道
    - Option1 :　下のようにモデルの一部にして、生の文字列を処理するモデルとする
        ```python
        text_input = keras.Input(shape=(1,), dtype=tf.string, name='text')
        x = vectorize_layer(text_input)
        x = layers.Embedding(max_features + 1, embedding_dim)(x)
        ...
        ```

    - Option2 :　テキストデータに適用し、単語のインデックスのデータセットを作る。それを整数列を入力するモデルに送り込む

> この2つの重要な違いは、オプション2では、GPUでトレーニングする際に、非同期のCPU処理とデータのバッファリングができることです。したがって、GPUでモデルをトレーニングする場合、おそらく最高のパフォーマンスを得るためにこのオプションを選択したいでしょう。以下、これを実行します。
  
>　モデルを本番環境にエクスポートする場合、上記のオプション1のコード・スニペットのように、生の文字列を入力として受け付けるモデルを出荷することになります。これはトレーニング後に行うことができます。これは最後のセクションで行います。
    

In [13]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

# データをベクトル化する
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

train_ds = train_ds.cache().prefetch(buffer_size=10)
val_ds = val_ds.cache().prefetch(buffer_size=10)
test_ds = test_ds.cache().prefetch(buffer_size=10)

#### Build a model
- ここでは埋め込み層から始まる単純な1次元コンボネットを用いる


In [15]:
# 語彙インデックスの整数入力
inputs = keras.Input(shape=(None,), dtype="int64")

# 次にこれらの語彙インデックスを次元空間にマッピングするレイヤーを追加する("embedding_dim"
x = layers.Embedding(max_features, embedding_dim)(inputs)
x = layers.Dropout(0.5)(x)

# Conv1D + global max pooling
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)

# 隠れ層
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

# 単一ユニット出力層に投影し、シグモイドにかける
predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)

model = keras.Model(inputs, predictions)

# Compile
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

#### Train the model

In [16]:
epochs = 3

model.fit(train_ds, validation_data=val_ds, epochs=epochs)

Epoch 1/3

625/625 [==============================] - 58s 91ms/step - loss: 0.4819 - accuracy: 0.7323 - val_loss: 0.2988 - val_accuracy: 0.8696
Epoch 2/3
625/625 [==============================] - 45s 73ms/step - loss: 0.2152 - accuracy: 0.9168 - val_loss: 0.3138 - val_accuracy: 0.8758
Epoch 3/3
625/625 [==============================] - 45s 73ms/step - loss: 0.1053 - accuracy: 0.9631 - val_loss: 0.3686 - val_accuracy: 0.8740


#### Evaluate the model on the test set

In [17]:
model.evaluate(test_ds)

782/782 [==============================] - 43s 54ms/step - loss: 0.4033 - accuracy: 0.8629


[0.4033046066761017, 0.8629199862480164]